In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sea
from scipy import stats
import matplotlib.pyplot as plt
from housing_elements import utils, los_altos_permits, san_francisco_permits, san_jose_permits, map_utils
from pathlib import Path

In [2]:
from importlib import reload
reload(los_altos_permits)
reload(san_francisco_permits)
reload(utils)

<module 'housing_elements.utils' from '/Users/sidharthkapur/personal-workspace/housing-elements/housing_elements/utils.py'>

In [3]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [4]:
# Silence an annoying warning that I get when running pd.read_excel
import warnings
warnings.filterwarnings("ignore", message="Data Validation extension is not supported and will be removed")

# Load sites and permits DataFrames for each city

In [5]:
sites_df = utils.load_all_sites()

In [6]:
all_cities = sites_df.jurisdict.unique()

In [7]:
len(all_cities)

108

In [8]:
%%time
cities_with_sites = {}

for city in all_cities:
    with HiddenPrints():
        try:
            sites = utils.load_site_inventory(city)
            assert sites.shape[0]
            cities_with_sites[city] = sites
            cities_with_sites[city].relcapcty.sum()
        except Exception:
            print(city, file=sys.stderr)

Orinda
Foster City


CPU times: user 12.4 s, sys: 291 ms, total: 12.7 s
Wall time: 12.8 s


In [9]:
len(cities_with_sites)

106

In [10]:
%%time
cities_with_permits = {}

for city in all_cities:
    with HiddenPrints():
        try:
            cities_with_permits[city] = utils.load_all_new_building_permits(city)
        except Exception:
            print(city, file=sys.stderr)

/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
Contra Costa County
Alameda County
Marin County
Saint Helena
Napa County
/Users/sidharthkapur/Library/Caches/pypoetry/virtualenvs/housing-elements-H302xxA9-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
Hillsborough
San Mateo County
Sonoma County


CPU times: user 7min 3s, sys: 20.3 s, total: 7min 24s
Wall time: 8min 28s


Solano County


In [11]:
len(cities_with_permits)

99

In [12]:
len(set(cities_with_permits).intersection(set(cities_with_sites)))

97

## Dump match results to JSON, for use in website

In [15]:
reload(utils)
reload(map_utils)
map_utils.write_matches_to_files(
    cities_with_sites,
    cities_with_permits,
    Path('./map_results')
)

100%|██████████| 106/106 [04:12<00:00,  2.39s/it]


## Data Cleaning Factoids

### Realistic Capacity

#### Pre-Existing Nulls

Two ways to calculate NA realistic capacities come to the same conclusion

In [ ]:
rhna5 = sites_df[sites_df.rhnacyc == 'RHNA5']

In [ ]:
rhna5.relcapcty[rhna5.relcapcty.isna()].shape

In [ ]:
rhna5[(rhna5.relcapcty.str.contains("/^([^0-9]*)$/") != False)].shape

In [ ]:
rhna5.shape

In [ ]:
2910 / rhna5.shape[0]

### Proportion of nulls that are our fault versus theirs

In [ ]:
for city, site in cities_with_sites.items():
    num_fail_parse_realcap = site.realcap_parse_fail.sum()
    if num_fail_parse_realcap:
        print(city, end='\t')
        print('Realcap', num_fail_parse_realcap)

In [ ]:
sum([site.realcap_parse_fail.sum() for city, site in cities_with_sites.items()])

None of the realistic capacity nulls are our fault.

#### Degenerate Rows

In [ ]:
no_realcap = pd.to_numeric(rhna5.relcapcty, errors='coerce') == 0
no_locacreage = pd.to_numeric(rhna5.locacres, errors='coerce') == 0
no_gacreage = pd.to_numeric(rhna5.gacres, errors='coerce') == 0

In [ ]:
rhna5[no_realcap & no_locacreage].shape

In [ ]:
rhna5[no_gacreage & no_locacreage].shape

In [ ]:
rhna5[no_gacreage].shape

I wouldn't say there are any degenerate rows. Even when locacreage is zero, we can rely on gacreage instead. 

There are, however, 901 sites with no realistic capacity and no allowable density. What kind of sites are these?

They're mostly in San Francisco and have a zoning type of PDR. According to SF's website, "In order to protect PDR, residential development would be prohibited, while office, retail, and institutional uses (schools, hospitals, etc.) would be limited. HOWEVER, residences, offices and retail which currently exist legally in these areas may stay indefinitely."


In [ ]:
sf_sites = cities_with_sites['San Francisco']
sf_sites_fixed = sf_sites[
    sf_sites['relcapcty'] != 0
]
cities_with_sites['San Francisco'] = sf_sites_fixed

In [ ]:
sf_sites.shape

## Results

In [ ]:
def get_results_for_city(city: str, match_by: str) -> pd.DataFrame:
    permits = cities_with_permits[city]
    sites = cities_with_sites[city]
    
    nonvacant_matches, nonvacant_sites, nonvacant_ratio = utils.calculate_pdev_for_nonvacant_sites(sites, permits, match_by)
    vacant_matches, vacant_sites, vacant_ratio = utils.calculate_pdev_for_vacant_sites(sites, permits, match_by)
    all_matches, all_sites, all_ratio = utils.calculate_pdev_for_inventory(sites, permits, match_by)
    
    return {
        'City': city,
        'Ratio of development on inventory sites': utils.calculate_total_units_permitted_over_he_capacity(sites, permits),
        'Mean overproduction': utils.calculate_underproduction_on_sites(sites, permits),
        'Inventory housing / all housing': utils.calculate_inventory_housing_over_all_housing(sites, permits),
        'P(dev) for nonvacant sites': nonvacant_ratio,
        'P(dev) for vacant sites': vacant_ratio,
        'P(dev) for inventory': all_ratio,
        '# nonvacant matches': f'{nonvacant_matches} / {nonvacant_sites}',
        '# vacant matches': f'{vacant_matches} / {vacant_sites}',
        '# matches': f'{all_matches} / {all_sites}',
    }

In [ ]:
cities = [city for city in all_cities if city in cities_with_permits and city in cities_with_sites]

In [ ]:
len(cities)

In [ ]:
%%time
with HiddenPrints():
    results_df = pd.DataFrame([get_results_for_city(city, match_by='apn') for city in cities])

In [ ]:
results_df[results_df['P(dev) for inventory'].isna()]

In [ ]:
sites_df[sites_df.jurisdict == 'Orinda']

In [ ]:
for city, permits in cities_with_permits.items():
    print(city, '\t', round(utils.fraction_apns_nan(permits), 2))

In [ ]:
cities_with_permits['Dublin'].apn.isna().mean()

In [ ]:
x = results_df["P(dev) for inventory"]
y = [utils.fraction_apns_nan(p) for c, p in cities_with_permits.items() if c in cities_with_sites]
plt.scatter(x, y, alpha=.2)
plt.xlabel("P(dev)")
plt.ylabel("% APNs NAN")
plt.title("Are missing APNs skewing our results?")

In [ ]:
%%time
with HiddenPrints():
    results_geo_df = pd.DataFrame([get_results_for_city(city, match_by='geo') for city in cities])

In [ ]:
reload(utils)

In [ ]:
%%time
with HiddenPrints():
    results_geo_lax_df = pd.DataFrame([get_results_for_city(city, match_by='geo') for city in cities])

In [ ]:
query = """
City in ['Mountain View', 'San Francisco']
"""

In [ ]:
results_geo_lax_df.query(query)

In [ ]:
results_geo_df.query(query)

In [ ]:
%%time
with HiddenPrints():
    results_both_df = pd.DataFrame([get_results_for_city(city, match_by='both') for city in cities])

In [ ]:
results_both_df.round(3)

### Histograms of QOIs

In [ ]:
results_df[results_df['P(dev) for inventory'] == 0].City.values

In [ ]:
reload(utils)

In [ ]:
utils.map_qoi('P(dev) for vacant sites', results_both_df)

In [ ]:
utils.map_qoi('P(dev) for nonvacant sites', results_both_df)

In [ ]:
utils.map_qoi('Ratio of development on inventory sites', results_both_df)

In [ ]:
utils.map_qoi('Mean overproduction', results_both_df)

In [ ]:
utils.map_qoi('Ratio of development on inventory sites', results_both_df)

In [ ]:
sea_plot = sea.histplot(results_both_df['P(dev) for nonvacant sites']).set_title("Each city's P(dev) for nonvacant sites")
sea_plot.get_figure().savefig('./figures/Pdev_nonvacant.png')

In [ ]:
sea_plot = sea.histplot(results_both_df['P(dev) for vacant sites']).set_title("Each city's P(dev) for vacant sites")
sea_plot.get_figure().savefig('./figures/Pdev_vacant.png')

In [ ]:
sea_plot = sea.histplot(results_both_df['P(dev) for vacant sites']).set_title("Each city's P(dev)")
sea_plot.get_figure().savefig('./figures/Pdev.png')

In [ ]:
sea_plot = sea.histplot(results_both_df['Ratio of development on inventory sites']).set_title("Each city's 1 / P(inventory|dev)")
sea_plot.get_figure().savefig('./figures/ratio_of_dev.png')

In [ ]:
sea_plot = sea.histplot(results_both_df['Mean overproduction']).set_title("Each city's mean overproduction")
sea_plot.get_figure().savefig('./figures/mean_overproduction.png')

In [ ]:
sea_plot = sea.histplot(results_both_df['Inventory housing / all housing']).set_title("Each city's Inventory housing / all housing")
sea_plot.get_figure().savefig('./figures/inventory_housing_over_all_housing.png')


### Did RHNA success in last cycle actually have anything to do with how good the site inventory was?

In [ ]:
rhna_success = results_both_df['Ratio of development on inventory sites']
p_dev = results_both_df['P(dev) for inventory']

is_null = np.isnan(rhna_success) | np.isnan(p_dev)
stats.pearsonr(rhna_success[~is_null], p_dev[~is_null])

In [ ]:
sea_plot = sea.scatterplot(x=rhna_success[~is_null], y=p_dev[~is_null])
sea_plot.set_title("Does RHNA success have anything to do with the realistic capacity of the inventory sites?")
sea_plot.get_figure().savefig('./figures/did_realistic_capacity_calcs_matter.png')

### Table of top 10 cities by P(dev) & bottom 10

#### Worst

In [ ]:
results_both_df.sort_values(by='P(dev) for inventory').head(10)

#### Best

In [ ]:
results_both_df.sort_values(by='P(dev) for inventory', ascending=False).head(10)

In [ ]:
combined_df = results_df.merge(
    results_geo_df,
    on='City',
    suffixes=[' (by APN)', ' (by geomatching)']
)

In [ ]:
combined_df.round(3)

In [ ]:
cities_sorted = results_both_df.sort_values('P(dev) for inventory', ascending=False)['City'].values
plt.figure(figsize=(20, 6))
ax = sea.barplot(x="City", y="P(dev) for inventory", orient="vertical",
            data=results_both_df, saturation=.5,
            order=cities_sorted)
ax.tick_params(axis='x', labelrotation=90)
plt.savefig(f'figures/pdev_by_city.jpg')

In [ ]:
all_df = combined_df.merge(
    results_both_df,
    on='City',
    suffixes=['', ' union']
)

In [ ]:
reload(utils)
utils.catplot_qoi(all_df, 'P(dev) for inventory', cities_sorted)

In [ ]:
results_df.to_csv('results/apn_matching_results.csv')
results_geo_df.to_csv('results/geo_matching_results.csv')
combined_df.to_csv('results/combined_df.csv')
results_both_df.to_csv('results/apn_or_geo_matching_results.csv')

Let's look into some cities where the APN results and Geo results are very out of line:

In [ ]:
combined_df[
    (combined_df['P(dev) for nonvacant sites (by APN)'] - combined_df['P(dev) for nonvacant sites (by geomatching)']).abs() > 0.3
]

In [ ]:
for column in ['P(dev) for nonvacant sites', 'P(dev) for vacant sites', 'P(dev) for inventory']:
    print(column, (combined_df[column + ' (by geomatching)'] >= combined_df[column + ' (by APN)']).mean())

In [ ]:
for column in ['P(dev) for nonvacant sites', 'P(dev) for vacant sites', 'P(dev) for inventory']:
    print(column, (combined_df[column + ' (by geomatching)'] == combined_df[column + ' (by APN)']).mean())

# "Ground truth" data

In [ ]:
def get_ground_truth_results_for_city(city: str) -> pd.DataFrame:
    if city == 'San Jose':
        permits = san_jose_permits.load_all_permits()
    elif city == 'San Francisco':
        permits = san_francisco_permits.load_all_permits()
    elif city == 'Los Altos':
        permits = los_altos_permits.load_all_permits()
    else:
        raise ValueError(f"Ground truth data not available for {city}")
        
    permits = utils.load_all_new_building_permits(city)
    sites = utils.load_site_inventory(city)
    
    return {
        'City': city,
        'Ratio of development on inventory sites': utils.calculate_total_units_permitted_over_he_capacity(sites, permits),
        'Mean overproduction': utils.calculate_underproduction_on_sites(sites, permits),
        'Inventory housing / all housing': utils.calculate_inventory_housing_over_all_housing(sites, permits),
        'P(dev) for nonvacant sites': utils.calculate_pdev_for_nonvacant_sites(sites, permits),
        'P(dev) for vacant sites': utils.calculate_pdev_for_vacant_sites(sites, permits),
        'P(dev) for inventory': utils.calculate_pdev_for_inventory(sites, permits),
    }

In [ ]:
permits = pd.read_csv("./data/raw_data/sf_permits.csv")
print(permits.columns)
date_cols = [c for c in permits.columns if 'Date' in c]
permits[date_cols] = permits[date_cols].apply(pd.to_datetime)

In [ ]:
print(permits.columns)

In [ ]:
%%time
ground_truth_cities = ['Los Altos', 'San Francisco', 'San Jose']
ground_truth_results_df = pd.DataFrame([get_ground_truth_results_for_city(city) for city in ground_truth_cities])

In [ ]:
ground_truth_results_df

# Additional summary stats for results section

In [ ]:
for col in ['# matches', '# nonvacant matches', '# vacant matches']:
    values = results_both_df[col]
    num = values.str.split('/').apply(lambda x: int(x[0]))
    denom = values.str.split('/').apply(lambda x: int(x[1]))
    print(col)
    print('{:.1%}'.format(num.sum() / denom.sum()))
    print()

In [ ]:
for col in ['P(dev) for vacant sites', 'P(dev) for nonvacant sites', 'P(dev) for inventory']:
    values = results_both_df[col]
    print(col)
    print('Median: {:.1%}'.format(8/5 * values.median()))
    print('Mean: {:.1%}'.format(8/5 * values.mean()))
    display(
        pd.cut(
            8/5 * values, 
            [0, 0.1, 0.25, 0.5, 1],
            right=False
        ).value_counts()
    )
    print()